# U-Net
U-Net is a semantic segmentation method proposed in U-Net: Convolutional Networks for Biomedical Image Segmentation presented at MICCAI (Medical Image Computing and Computer-Assisted Intervention) 2015. Semantic Segmentation is a method proposed in MICCAI (Medical Image Computing and Computer-Assisted Intervention) 2015.

Semantic Segmentation is a deep learning method that labels each pixel in an image. As the name implies, U-Net has a U-shaped network structure.

U-Net does not have a total coupling layer, but is a network composed of convolutional layers. U-Net has an almost symmetric Encoder-Decoder structure, where the down-sampled feature maps are up-sampled by the Decoder through pooling of the Encoder. U-Net introduces concatenation of the feature maps output at each layer of the Encoder to the corresponding feature maps at each layer of the Decoder, which is called a skip connection.

In this project, we will use deep learning to extract buildings from satellite images. The procedure is as follows:

(1) Preparation of dataset

(2) Learning of U-Net

(3) Evaluation of U-Net

## GPU environment

In [1]:
!nvidia-smi

Sat Apr 20 05:55:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Apr 20 05:55:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0              26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


## データセットの準備
まずは、 SpaceNetデータセットからデータをダウンロードするために `pip` で AWS Command Line Interface（AWS CLI） をインストールします。 SpaceNetは、アメリカのDigitalGlobe社が保有している複数の人工衛星が撮影した画像と建物や道路道路といった特定の地物のラベルがセットになったデータセットです。


In [4]:
!pip install awscli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1


AWSのS3内にあるSpaceNetのデータをダウンロードします。次にダウンロードしたデータを解凍します。解凍後のディレクトリ名は、trainとしました。今回は、“SpaceNet 1: Building Detection v1”のデータセットをダウンロードしました。

In [5]:
!aws s3 cp s3://spacenet-dataset/spacenet/SN1_buildings/tarballs/SN1_buildings_train_AOI_1_Rio_3band.tar.gz . --no-sign-request
!mkdir train && tar -zxvf SN1_buildings_train_AOI_1_Rio_3band.tar.gz -C train --strip-components 1

Streaming output truncated to the last 5000 lines.
3band/3band_AOI_1_RIO_img5153.tif
3band/3band_AOI_1_RIO_img6252.tif
3band/3band_AOI_1_RIO_img983.tif
3band/3band_AOI_1_RIO_img3667.tif
3band/3band_AOI_1_RIO_img579.tif
3band/3band_AOI_1_RIO_img6631.tif
3band/3band_AOI_1_RIO_img5020.tif
3band/3band_AOI_1_RIO_img6736.tif
3band/3band_AOI_1_RIO_img1889.tif
3band/3band_AOI_1_RIO_img4499.tif
3band/3band_AOI_1_RIO_img6485.tif
3band/3band_AOI_1_RIO_img4307.tif
3band/3band_AOI_1_RIO_img306.tif
3band/3band_AOI_1_RIO_img6435.tif
3band/3band_AOI_1_RIO_img2606.tif
3band/3band_AOI_1_RIO_img5130.tif
3band/3band_AOI_1_RIO_img2018.tif
3band/3band_AOI_1_RIO_img5490.tif
3band/3band_AOI_1_RIO_img1879.tif
3band/3band_AOI_1_RIO_img1904.tif
3band/3band_AOI_1_RIO_img6530.tif
3band/3band_AOI_1_RIO_img4167.tif
3band/3band_AOI_1_RIO_img4489.tif
3band/3band_AOI_1_RIO_img5521.tif
3band/3band_AOI_1_RIO_img6155.tif
3band/3band_AOI_1_RIO_img1820.tif
3band/3band_AOI_1_RIO_img3823.tif
3band/3band_AOI_1_RIO_img3859.tif


次に、建物の位置が記されたファイルをダウンロードします。 解凍後のディレクトリ名は、geojsonとしました。

In [6]:
!aws s3 cp s3://spacenet-dataset/spacenet/SN1_buildings/tarballs/SN1_buildings_train_AOI_1_Rio_geojson_buildings.tar.gz . --no-sign-request
!mkdir geojson && tar -zxvf SN1_buildings_train_AOI_1_Rio_geojson_buildings.tar.gz -C geojson --strip-components 1

Streaming output truncated to the last 5000 lines.
geojson/Geo_AOI_1_RIO_img1820.geojson
geojson/Geo_AOI_1_RIO_img3330.geojson
geojson/Geo_AOI_1_RIO_img6319.geojson
geojson/Geo_AOI_1_RIO_img3604.geojson
geojson/Geo_AOI_1_RIO_img1910.geojson
geojson/Geo_AOI_1_RIO_img783.geojson
geojson/Geo_AOI_1_RIO_img1311.geojson
geojson/Geo_AOI_1_RIO_img3550.geojson
geojson/Geo_AOI_1_RIO_img4303.geojson
geojson/Geo_AOI_1_RIO_img2340.geojson
geojson/Geo_AOI_1_RIO_img2388.geojson
geojson/Geo_AOI_1_RIO_img1324.geojson
geojson/Geo_AOI_1_RIO_img1477.geojson
geojson/Geo_AOI_1_RIO_img5827.geojson
geojson/Geo_AOI_1_RIO_img2262.geojson
geojson/Geo_AOI_1_RIO_img1570.geojson
geojson/Geo_AOI_1_RIO_img332.geojson
geojson/Geo_AOI_1_RIO_img1524.geojson
geojson/Geo_AOI_1_RIO_img1359.geojson
geojson/Geo_AOI_1_RIO_img6305.geojson
geojson/Geo_AOI_1_RIO_img5068.geojson
geojson/Geo_AOI_1_RIO_img269.geojson
geojson/Geo_AOI_1_RIO_img4395.geojson
geojson/Geo_AOI_1_RIO_img1888.geojson
geojson/Geo_AOI_1_RIO_img6553.geojson
ge

最後に、テストデータをダウンロードし、先ほどと同様に解凍します。解凍後のディレクトリ名は、testとしました。

In [7]:
!aws s3 cp s3://spacenet-dataset/spacenet/SN1_buildings/tarballs/SN1_buildings_test_AOI_1_Rio_3band.tar.gz . --no-sign-request
!mkdir test && tar -zxvf SN1_buildings_test_AOI_1_Rio_3band.tar.gz -C test --strip-components 1

download: s3://spacenet-dataset/spacenet/SN1_buildings/tarballs/SN1_buildings_test_AOI_1_Rio_3band.tar.gz to ./SN1_buildings_test_AOI_1_Rio_3band.tar.gz
3band/3band_AOI_2_RIO_img1645.tif
3band/3band_AOI_2_RIO_img2794.tif
3band/3band_AOI_2_RIO_img981.tif
3band/3band_AOI_2_RIO_img2710.tif
3band/3band_AOI_2_RIO_img2261.tif
3band/3band_AOI_2_RIO_img23.tif
3band/3band_AOI_2_RIO_img2611.tif
3band/3band_AOI_2_RIO_img800.tif
3band/3band_AOI_2_RIO_img1623.tif
3band/3band_AOI_2_RIO_img407.tif
3band/3band_AOI_2_RIO_img790.tif
3band/3band_AOI_2_RIO_img1891.tif
3band/3band_AOI_2_RIO_img460.tif
3band/3band_AOI_2_RIO_img690.tif
3band/3band_AOI_2_RIO_img1198.tif
3band/3band_AOI_2_RIO_img715.tif
3band/3band_AOI_2_RIO_img1859.tif
3band/3band_AOI_2_RIO_img678.tif
3band/3band_AOI_2_RIO_img2534.tif
3band/3band_AOI_2_RIO_img1501.tif
3band/3band_AOI_2_RIO_img1152.tif
3band/3band_AOI_2_RIO_img2490.tif
3band/3band_AOI_2_RIO_img1484.tif
3band/3band_AOI_2_RIO_img1088.tif
3band/3band_AOI_2_RIO_img77.tif
3band/3ba

### U-Netモデルの学習
次に、学習の準備をしていきます。まずは、学習のために画像と建物の位置情報を対応付けます。

In [9]:
import os
from tqdm import tqdm
from osgeo import gdal, ogr
from PIL import Image
import numpy as np

def create_poly_mask(rasterSrc, vectorSrc, npDistFileName='', noDataValue=0, burn_values=1):
  # read the location information of building
  source_ds = ogr.Open(vectorSrc)
  source_layer = source_ds.GetLayer()

  # read the satellite image
  srcRas_ds = gdal.Open(rasterSrc)
  cols = srcRas_ds.RasterXSize
  rows = srcRas_ds.RasterYSize

  if npDistFileName == '':
    dstPath = ".tmp.tiff"
  else:
    dstPath = npDistFileName

  memdrv = gdal.GetDriverByName("GTiff")
  dst_ds = memdrv.Create(dstPath, cols, rows, 1, gdal.GDT_Byte, options=["COMPRESS=LZW"])
  dst_ds.SetGeoTransform(srcRas_ds.GetGeoTransform())
  dst_ds.SetProjection(srcRas_ds.GetProjection())
  band = dst_ds.GetRasterBand(1)
  band.SetNoDataValue(noDataValue)
  gdal.RasterizeLayer(dst_ds, [1], source_layer, burn_values=[burn_values])
  dst_ds = 0

  #Masked satellite image with building location information
  mask_image = Image.open(dstPath)
  mask_image = np.array(mask_image)
  if npDistFileName == '':
    os.remove(dstPath)

  return mask_image

def build_labels(src_raster_dir, src_vector_dir, dst_dir):
  os.makedirs(dst_dir, exist_ok=True)
  file_count = len([f for f in os.walk(src_vector_dir).__next__()[2] if f[-8:] == ".geojson"])
  print(file_count)
  for idx in tqdm(range(1, file_count + 1)):
    src_raster_filename = "3band_AOI_1_RIO_img{}.tif".format(idx)
    src_vector_filename = "Geo_AOI_1_RIO_img{}.geojson".format(idx)
    src_raster_path = os.path.join(src_raster_dir, src_raster_filename)
    src_vector_path = os.path.join(src_vector_dir, src_vector_filename)
    dst_path = os.path.join(dst_dir, src_raster_filename)
    create_poly_mask(src_raster_path, src_vector_path, npDistFileName=dst_path, noDataValue=0, burn_values=255)

if __name__ == "__main__":
  src_raster_dir = "train" # Root directory for raster files (.tif)
  src_vector_dir = "geojson" # Root directory for vector files (.geojson)
  dst_dir = "buildingMaskImages" # Output directory
  build_labels(src_raster_dir, src_vector_dir, dst_dir)


6940


100%|██████████| 6940/6940 [00:44<00:00, 156.73it/s]


次にU-NetをPythonで実装した`unet`を`pip`でインストールします。

In [10]:
!pip install unet
!pip install keras

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

最後にU-Netで学習します。

In [11]:
import os
import cv2
import glob
import numpy as np
import keras.backend as K
from keras.models import Model
from keras.layers import Conv2D, ZeroPadding2D, Conv2DTranspose
from keras.layers import concatenate
from keras.layers import LeakyReLU, BatchNormalization, Activation, Dropout, Input
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from unet import UNet

In [12]:
class UNet(object):
  def __init__(self, input_channel_count, output_channel_count, first_layer_filter_count):
    self.INPUT_IMAGE_SIZE = 256
    self.CONCATENATE_AXIS = -1
    self.CONV_FILTER_SIZE = 4
    self.CONV_STRIDE = 2
    self.CONV_PADDING =(1,1)
    self.DECONV_FILTER_SIZE = 2
    self.DECONV_STRIDE = 2

    # (256 x 256 x input_channel_count)
    inputs = Input((self.INPUT_IMAGE_SIZE, self.INPUT_IMAGE_SIZE, input_channel_count))

    # Generate encoder
    # (128 x 128 x N)
    enc1 = ZeroPadding2D(self.CONV_PADDING)(inputs)
    enc1 = Conv2D(first_layer_filter_count, self.CONV_FILTER_SIZE, strides=self.CONV_STRIDE)(enc1)

    # (64 x 64 x 2N)
    filter_count = first_layer_filter_count*2
    enc2 = self._add_encoding_layer(filter_count, enc1)

    # (32 x 32 x 4N)
    filter_count = first_layer_filter_count*4
    enc3 = self._add_encoding_layer(filter_count, enc2)

    # (16 x 16 x 8N)
    filter_count = first_layer_filter_count*8
    enc4 = self._add_encoding_layer(filter_count, enc3)

    # (8 x 8 x 8N)
    enc5 = self._add_encoding_layer(filter_count, enc4)

    # (4 x 4 x 8N)
    enc6 = self._add_encoding_layer(filter_count, enc5)

    # (2 x 2 x 8N)
    enc7 = self._add_encoding_layer(filter_count, enc6)

    # (1 x 1 x 8N)
    enc8 = self._add_encoding_layer(filter_count, enc7)

    # Generate decode
    # (2 x 2 x 8N)
    dec1 = self._add_decoding_layer(filter_count, True, enc8)
    dec1 = concatenate([dec1, enc7], axis=self.CONCATENATE_AXIS)

    # (4 x 4 x 8N)
    dec2 = self._add_decoding_layer(filter_count, True, dec1)
    dec2 = concatenate([dec2, enc6], axis=self.CONCATENATE_AXIS)

    # (8 x 8 x 8N)
    dec3 = self._add_decoding_layer(filter_count, True, dec2)
    dec3 = concatenate([dec3, enc5], axis=self.CONCATENATE_AXIS)

    # (16 x 16 x 8N)
    dec4 = self._add_decoding_layer(filter_count, False, dec3)
    dec4 = concatenate([dec4, enc4], axis=self.CONCATENATE_AXIS)

    # (32 x 32 x 4N)
    filter_count = first_layer_filter_count*4
    dec5 = self._add_decoding_layer(filter_count, False, dec4)
    dec5 = concatenate([dec5, enc3], axis=self.CONCATENATE_AXIS)

    # (64 x 64 x 2N)
    filter_count = first_layer_filter_count*2
    dec6 = self._add_decoding_layer(filter_count, False, dec5)
    dec6 = concatenate([dec6, enc2], axis=self.CONCATENATE_AXIS)

    # (128 x 128 x N)
    filter_count = first_layer_filter_count
    dec7 = self._add_decoding_layer(filter_count, False, dec6)
    dec7 = concatenate([dec7, enc1], axis=self.CONCATENATE_AXIS)

    # (256 x 256 x output_channel_count)
    dec8 = Activation(activation="relu")(dec7)
    dec8 = Conv2DTranspose(output_channel_count, self.DECONV_FILTER_SIZE, strides=self.DECONV_STRIDE)(dec8)
    dec8 = Activation(activation="sigmoid")(dec8)

    self.UNET = Model(input=inputs, output=dec8)

  def _add_encoding_layer(self, filter_count, sequence):
    new_sequence = LeakyReLU(0.2)(sequence)
    new_sequence = ZeroPadding2D(self.CONV_PADDING)(new_sequence)
    new_sequence = Conv2D(filter_count,self.CONV_FILTER_SIZE, strides=self.CONV_STRIDE)(new_sequence)
    new_sequence = BatchNormalization()(new_sequence)
    return new_sequence

  def _add_decoding_layer(self, filter_count, add_drop_layer, sequence):
    new_sequence = Activation(activation="relu")(sequence)
    new_sequence = Conv2DTranspose(filter_count, self.DECONV_FILTER_SIZE, strides=self.DECONV_STRIDE, kernel_initializer="he_uniform")(new_sequence)
    new_sequence = BatchNormalization()(new_sequence)
    if add_drop_layer:
      new_sequence = Dropout(0.5)(new_sequence)
    return new_sequence

  def get_model(self):
    return self.UNET

IMAGE_SIZE = 256

# 値を-1から1に正規化する関数
def normalize_x(image):
  image = image/127.5 - 1
  return image

# 値を0から1に正規化する関数
def normalize_y(image):
  image = image/255
  return image

# 値を0から255に戻す関数
def denormalize_y(image):
  image = image*255
  return image

# インプット画像を読み込む関数
def load_X(folder_path):
  image_files = glob.glob(folder_path + "/*.tif")
  image_files.sort()
  images = np.zeros((len(image_files), IMAGE_SIZE, IMAGE_SIZE, 3), np.float32)
  for i, image_file in enumerate(image_files):
    image = cv2.imread(image_file, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    images[i] = normalize_x(image)
  return images, image_files

# ラベル画像を読み込む関数
def load_Y(folder_path):
  image_files = glob.glob(folder_path + "/*.tif")
  image_files.sort()
  images = np.zeros((len(image_files), IMAGE_SIZE, IMAGE_SIZE, 1), np.float32)
  for i, image_file in enumerate(image_files):
    image = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = image[:, :, np.newaxis]
    images[i] = normalize_y(image)
  return images

# ダイス係数を計算する関数
def dice_coef(y_true, y_pred):
  y_true = K.flatten(y_true)
  y_pred = K.flatten(y_pred)
  intersection = K.sum(y_true * y_pred)
  return 2.0 * intersection / (K.sum(y_true) + K.sum(y_pred) + 1)

# ロス関数
def dice_coef_loss(y_true, y_pred):
  return 1.0 - dice_coef(y_true, y_pred)

# U-Netのトレーニングを実行する関数
def train_unet():
  # 衛星画像を置いている場所を指定
  X_train, file_names = load_X("train")
  # mask画像を置いている場所を指定
  Y_train = load_Y("buildingMaskImages")

  # 入力はBGR3チャンネル
  input_channel_count = 3
  # 出力はグレースケール1チャンネル
  output_channel_count = 1
  # 一番初めのConvolutionフィルタ枚数は64
  first_layer_filter_count = 64
  # U-Netの生成
  network = UNet(input_channel_count, output_channel_count, first_layer_filter_count)
  model = network.get_model()
  model.compile(loss=dice_coef_loss, optimizer=Adam(), metrics=[dice_coef])

  BATCH_SIZE = 12
  NUM_EPOCH = 1
  history = model.fit(X_train, Y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCH, verbose=1)
  model.save_weights('unet_weights.hdf5')


## U-Netの評価
テストデータの衛星画像を用いて学習したU-Netでセグメンテーションを行います。


In [13]:
def predict():
  # testデータのある場所を指定
  X_test, file_names = load_X("test")

  input_channel_count = 3
  output_channel_count = 1
  first_layer_filter_count = 64
  network = UNet(input_channel_count, output_channel_count, first_layer_filter_count)
  model = network.get_model()
  model.load_weights("unet_weights.hdf5")
  BATCH_SIZE = 12
  Y_pred = model.predict(X_test, BATCH_SIZE)

  for i, y in enumerate(Y_pred):
    img = cv2.imread(file_names[i], cv2.IMREAD_COLOR)
    y = cv2.resize(y, (img.shape[1], img.shape[0]))
    img_2 = cv2.cvtColor(denormalize_y(y), cv2.COLOR_GRAY2BGR)
    left_img = np.array(img, dtype="int32")
    right_img = np.array(img_2, dtype="int32")
    result_img = cv2.hconcat([left_img, right_img])
    cv2.imwrite('result/prediction' + str(i) + '.png', result_img)

In [14]:
if __name__ == '__main__':
  train_unet()
  predict()

TypeError: ('Keyword argument not understood:', 'input')

セグメンテーションの結果は、ディレクトリresultに入っています。なお、今回はU-Netの繰り返しの学習回数（epoch数）を1回にしています。epoch数（NUM_EPOCH）を変えるとさらに精度が上がりますので試してみてください。